In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

In [ ]:
def watershed(image, Timage, index):
    gray = cv.cvtColor(image,cv.COLOR_BGR2GRAY)
    _, thresh = cv.threshold(gray,0,255,cv.THRESH_BINARY_INV+cv.THRESH_OTSU)

    kernel = np.ones((3,3),np.uint8)
    opening = cv.morphologyEx(thresh,cv.MORPH_OPEN,kernel, iterations = 2)

    sure_bg = cv.dilate(opening,kernel,iterations=3)

    dist_transform = cv.distanceTransform(opening,cv.DIST_L2,5)
    _, sure_fg = cv.threshold(dist_transform,0.7*dist_transform.max(),255,0)

    sure_fg = np.uint8(sure_fg)
    unknown = cv.subtract(sure_bg,sure_fg)

    _, markers = cv.connectedComponents(sure_fg)
    markers = markers+1
    markers[unknown==255] = 0

    markers = cv.watershed(image, markers)

    copy = np.zeros_like(image)
    copy[markers == -1] = [255,255,255]

    img = cv.cvtColor(copy, cv.COLOR_BGR2GRAY)
    
    cv.imwrite('./watershed_edges/' + str(index) + '.jpg', img)
    cv.imshow('dst', img)
    cv.waitKey(0)
    cv.destroyAllWindows()

    contours, _ = cv.findContours(img, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    # cnt = sorted(contours, key = cv.contourArea, reverse=True)
    cp = Timage.copy()

    t = cv.drawContours(cp, contours, -1, (255, 255, 255), 1)
    
    cv.imwrite('./watershed_thermal/' + str(index) + '.jpg', t)
    
    cv.imshow('dst', t)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [ ]:
def otsuThreshold(image, Timage, index):
    blurred = cv.blur(image, (3,3))
    img = cv.cvtColor(blurred, cv.COLOR_BGR2GRAY)
    _, th = cv.threshold(img, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
    
    cv.imshow('res', th)
    cv.waitKey(0)
    cv.destroyAllWindows()

    contours, _ = cv.findContours(th, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    copy = Timage.copy()
    black = np.zeros(Timage.shape)
    black_edges = cv.drawContours(black, contours, -1, (255,255,255), 1) 
    th_edges = cv.drawContours(copy, contours, -1, (255,255,255), 1)

    cv.imshow('edges', black_edges)
    cv.waitKey(0)
    cv.destroyAllWindows()

    cv.imshow('edges', th_edges)
    cv.waitKey(0)
    cv.destroyAllWindows()

    masked = cv.bitwise_and(Timage, Timage, mask=th)

    gray = cv.cvtColor(masked, cv.COLOR_BGR2GRAY)
    for i in gray:
        print(i)

    cv.imshow('masked', masked)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [ ]:
def GrabCut(image):
    
    bgdModel = np.zeros((1,65),np.float64)
    fgdModel = np.zeros((1,65),np.float64)

    mask = np.zeros(image.shape[:2],np.uint8)
    rect = (x,y,w,h)
    fg = cv.grabCut(image, mask, rect, bgdModel, fgdModel, 5, cv.GC_INIT_WITH_RECT)

    mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8')
    t = image*mask2[:,:,np.newaxis]

    cv.imshow('grabcut', t)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [ ]:
def CannyWithOtsu(image, Timage, index):
    blurred = cv.blur(image, (3, 3))
    # img = cv.cvtColor(blurred, cv.COLOR_BGR2GRAY)
    img = cv.cvtColor(blurred, cv.COLOR_BGR2GRAY)
    
    cv.imshow('blurred', img)
    cv.waitKey(0)
    cv.destroyAllWindows()

    res, th = cv.threshold(img, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
    # print(res)
    # cv.imwrite('./otsu_binary/' + str(index) + '.jpg', th)

    # cv.imshow('otsu', th)
    # cv.waitKey(0)
    # cv.destroyAllWindows()
    
    edges = cv.Canny(img, res*0.5, res)
    # edges = cv.dilate(edges, (5,5))

    cv.imshow('edges', edges)
    cv.waitKey(0)
    cv.destroyAllWindows()
    # cv.imwrite('./otsu_edges/' + str(index) + '.jpg', edges)


    contours, _ = cv.findContours(edges, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    # sort_cont = sorted(contours, key = cv.contourArea, reverse=True)
    # sort_cont = sorted(contours, key = lambda x : cv.arcLength(x, closed=False), reverse=True)

    # x,y,w,h = cv.boundingRect(cnt)
    copy = Timage.copy()
    copy2 = Timage.copy()
    # t = cv.rectangle(copy, (x,y), (x+w, y+h), (0,255,0), 1)

    # for i in range(len(contours)):
    #     t = cv.drawContours(copy2, contours, i, (255, 255, 255), 1)    
    t = cv.drawContours(copy, contours, -1, (255, 255, 255), 1)
    
    cv.imwrite('./otsu_thermal/' + str(index) + '.jpg', t)
    cv.imshow('contours', t)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [ ]:
def adaptThreshold(image, Timage, index):
    img = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    th = cv.adaptiveThreshold(img, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 13, 2)
    
    th = cv.bitwise_not(th)
    th = cv.morphologyEx(th, cv.MORPH_OPEN, (7,7), iterations=5)
    th = cv.morphologyEx(th, cv.MORPH_ERODE, (5,5), iterations=7)

    cv.imshow('res', th)
    cv.waitKey(0)
    cv.destroyAllWindows()

    contours, _ = cv.findContours(th, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    copy = Timage.copy()
    t = cv.drawContours(copy, contours, -1, (255, 255, 255), 1)

    cv.imshow('dst', t)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [ ]:
def ImgProcess(img, Timg, cropX, cropY, index):
    x1, x2 = cropX[index-1]
    y1, y2 = cropY[index-1]
    vis = img[y1:y2, x1:x2]
    th = Timg[y1:y2, x1:x2]

    cv.imwrite('./cropped/' + str(index) + '.jpg', vis)

    cv.imshow('Visual', vis)
    cv.waitKey(0)
    cv.destroyAllWindows()
    
    otsuThreshold(vis, th, index)

In [ ]:
cropX = [(5, 294), (5, 295), (11, 296), (5,293), (8, 292)]
cropY = [(24, 217), (27, 228), (10, 193), (35,210), (3, 218)]

for i in range(1,6):
    visual = './Assets/foreground/visual/' + str(i) + '.jpg'
    thermal = './Assets/foreground/thermal/' + str(i) + '.jpg'

    img = cv.imread(visual)
    Timg = cv.imread(thermal)

    ImgProcess(img, Timg, cropX, cropY, i)